In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import dash_daq as daq
import plotly.express as px
import pandas as pd
import xarray as xr
import datetime as dt
import numpy as np
import pytz
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

# App to check latest data (12 hours), app updates automatically every 5 minutes

lighthouses = ["Narveneset", "Bohemanneset", "Daudmannsodden", "Gasoyane", "KappThordsen"]
boats = ["MSPolargirl", "MSBillefjord", "MSBerg"]


app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

variables = ["Temperature [°C]", "Relative Humidity [%]", "Pressure [hPa]", "Wind Speed [m/s]", "Wind Direction [°]"]
variable_dropdown = html.Div(dcc.Dropdown(variables, "Wind Speed [m/s]", id="variable-dropdown"))
station_dropdown = html.Div(dcc.Dropdown(lighthouses, lighthouses, id="station-dropdown", multi=True), style={'padding': 10})
utc_switch = html.Div(daq.BooleanSwitch(id='utc-switch', on=False, label="UTC", labelPosition="top"))

map_if = dbc.Card(dcc.Graph(id="map"))
lh_timeseries = dbc.Card(dcc.Graph(id="lighthouse-ts"))

app.layout = dbc.Container([html.H2("Latest IWIN Overview", className="text-center"),
                            dbc.Row([dbc.Col(variable_dropdown),
                                     dbc.Col(utc_switch, width=1)]), 
                            dbc.Row(dbc.Col(map_if)),
                            dbc.Row(dbc.Col(station_dropdown)),
                            dbc.Row(dbc.Col(lh_timeseries))], 
                            fluid=True)

In [ ]:
@app.callback(
    Output(
)



@app.callback(
    Output('map', 'figure'),
    Output('lighthouse-ts', 'figure'),
    Input("station-dropdown", "value"),
    Input("variable-dropdown", "value"),
    Input("utc-switch", "on")
)
def update_graphs(lh_st, vari, switch_utc):
    if isinstance(lh_st, str):
        lh_st = [lh_st]
    
    if vari == None:
        dr = pd.date_range(lighthouse_data[list(lighthouse_data.keys())[0]].time[0].values,
                           lighthouse_data[list(lighthouse_data.keys())[0]].time[-1].values, freq="1min", name="time")
        df = pd.DataFrame(np.ones((len(dr), len(lh_st)))*np.nan, index=dr, columns=pd.Index(st, name="Station"))
        df.loc[:] = np.nan
        vari = "None"
    else:
        if len(lh_st) == 0:
            dr = pd.date_range(lighthouse_data[list(lighthouse_data.keys())[0]].time[0].values,
                           lighthouse_data[list(lighthouse_data.keys())[0]].time[-1].values, freq="1min", name="time")
            df = pd.DataFrame(np.ones((len(dr)))*np.nan, index=dr, columns=pd.Index([0], name="Station"))
            df.loc[:] = np.nan
        else:
            df = [lighthouse_data[s][vari].dropna("time").to_dataframe(name=s) for s in lh_st]
            df = pd.concat(df, axis=1)


    df.index = df.index.tz_localize(tz='UTC')

    time_label = "Time [UTC]"
    if not switch_utc:
        df.index = df.index.tz_convert("Europe/Oslo")
        time_label = "Local Time"
    
    if vari == "Wind Direction [°]":
        timezone = pytz.timezone("Europe/Oslo")
        aware = timezone.localize(dt.datetime.now())
        utc_os = aware.utcoffset()
        utc_os_wd = utc_os.seconds//3600
        fig_ts = px.scatter(df, labels={"value": vari, "time": time_label, "variable": "Station"}, range_x=[(pd.Timestamp(df.index[0]).value/1.e6)-utc_os_wd*(3600.e3), (pd.Timestamp(df.index[-1]).value/1.e6)-utc_os_wd*(3600.e3)])
        fig_ts.update_traces(marker={'size': 3})
        fig_ts.update_yaxes(range=[0., 360.], tickvals=[0., 45., 90., 135., 180., 225., 270., 315., 360.], ticktext=["N", "NE", "E", "SE", "S", "SW", "W", "NW", "N"],)
    else:
        fig_ts = px.line(df, y=st, labels={"time": time_label, "variable": "Station", "value": vari})
    
    
    return [fig_map, fig_ts]





if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# read lighthouse data from the last two days
lighthouse_data = {}

today = dt.date.today()
start = today - dt.timedelta(days=2)
end = today + dt.timedelta(days=1)


for lighthouse in lighthouses:
    print(lighthouse)
    with xr.open_dataset(f"https://thredds.met.no/thredds/dodsC/met.no/observations/unis/lighthouse_AWS_{lighthouse}_1min") as f:
        ds = f[["temperature","wind_speed","wind_direction","relative_humidity","air_pressure"]].sel(time=slice(start,end))
        ds = ds.assign_attrs(Longitude=f.attrs["longitude"], Latitude=f.attrs["latitude"])
        ds = ds.rename({"temperature": "Temperature [°C]",
                "relative_humidity": "Relative Humidity [%]",
                "air_pressure": "Pressure [hPa]",
                "wind_speed": "Wind Speed [m/s]",
                "wind_direction": "Wind Direction [°]"})
        lighthouse_data[lighthouse] = ds




@app.callback(
    Output('lighthouse-timeseries-plot', 'figure'),
    Input("station-dropdown", "value"),
    Input("variable-dropdown", "value"),
    Input("utc-switch", "on")
)
def update_graph(st, vari, switch_utc):
    if isinstance(st, str):
        st = [st]
    
    if vari == None:
        dr = pd.date_range(lighthouse_data[list(lighthouse_data.keys())[0]].time[0].values,
                           lighthouse_data[list(lighthouse_data.keys())[0]].time[-1].values, freq="1min", name="time")
        df = pd.DataFrame(np.ones((len(dr), len(st)))*np.nan, index=dr, columns=pd.Index(st, name="Station"))
        df.loc[:] = np.nan
        vari = "None"
    else:
        if len(st) == 0:
            dr = pd.date_range(lighthouse_data[list(lighthouse_data.keys())[0]].time[0].values,
                           lighthouse_data[list(lighthouse_data.keys())[0]].time[-1].values, freq="1min", name="time")
            df = pd.DataFrame(np.ones((len(dr)))*np.nan, index=dr, columns=pd.Index([0], name="Station"))
            df.loc[:] = np.nan
        else:
            df = [lighthouse_data[s][vari].dropna("time").to_dataframe(name=s) for s in st]
            df = pd.concat(df, axis=1)


    df.index = df.index.tz_localize(tz='UTC')

    time_label = "Time [UTC]"
    if not switch_utc:
        df.index = df.index.tz_convert("Europe/Oslo")
        time_label = "Local Time"
    
    if vari == "Wind Direction [°]":
        timezone = pytz.timezone("Europe/Oslo")
        aware = timezone.localize(dt.datetime.now())
        utc_os = aware.utcoffset()
        utc_os_wd = utc_os.seconds//3600
        fig = px.scatter(df, labels={"value": vari, "time": time_label, "variable": "Station"}, range_x=[(pd.Timestamp(df.index[0]).value/1.e6)-utc_os_wd*(3600.e3), (pd.Timestamp(df.index[-1]).value/1.e6)-utc_os_wd*(3600.e3)])
        fig.update_traces(marker={'size': 3})
        fig.update_yaxes(range=[0., 360.], tickvals=[0., 45., 90., 135., 180., 225., 270., 315., 360.], ticktext=["N", "NE", "E", "SE", "S", "SW", "W", "NW", "N"],)
    else:
        fig = px.line(df, y=st, labels={"time": time_label, "variable": "Station", "value": vari})
    return fig

if __name__ == '__main__':
    app.run(debug=True)